<a href="https://colab.research.google.com/github/Karuni001/Cognify_ML/blob/main/task2_Restaurant_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#Loading and clean the dataset
df = pd.read_csv("restaurant_data.csv")

df = df[['Restaurant Name', 'City', 'Cuisines', 'Average Cost for two', 'Price range', 'Has Online delivery',
         'Has Table booking']]
df.dropna(inplace=True)
print("Columns in dataset:", df.columns.tolist())
print("Initial rows:\n", df.head())

Columns in dataset: ['Restaurant Name', 'City', 'Cuisines', 'Average Cost for two', 'Price range', 'Has Online delivery', 'Has Table booking']
Initial rows:
           Restaurant Name              City                          Cuisines  \
0        Le Petit Souffle       Makati City        French, Japanese, Desserts   
1        Izakaya Kikufuji       Makati City                          Japanese   
2  Heat - Edsa Shangri-La  Mandaluyong City  Seafood, Asian, Filipino, Indian   
3                    Ooma  Mandaluyong City                   Japanese, Sushi   
4             Sambo Kojin  Mandaluyong City                  Japanese, Korean   

   Average Cost for two  Price range Has Online delivery Has Table booking  
0                  1100            3                  No               Yes  
1                  1200            3                  No               Yes  
2                  4000            4                  No               Yes  
3                  1500            4           

In [5]:
df["Average Cost for two"] = df["Average Cost for two"].astype(str).str.replace(",", "")
df["Average Cost for two"] = pd.to_numeric(df["Average Cost for two"], errors="coerce")
df.dropna(inplace=True)

df['Cuisines'] = df['Cuisines'].astype(str).str.lower()
df['City'] = df['City'].astype(str).str.lower()

In [6]:
df['combined'] = df['Cuisines'] + " " + df['City'] + " " + df['Price range'].astype(str)

In [8]:
#Converting to TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [9]:
# Recommendation function
def recommend_restaurants(user_cuisine, user_city, user_price_range, top_n=5):
    user_pref = f"{user_cuisine.lower()} {user_city.lower()} {user_price_range}"
    user_vec = tfidf.transform([user_pref])

    sim_scores = cosine_similarity(user_vec, tfidf_matrix).flatten()
    top_indices = sim_scores.argsort()[-top_n:][::-1]

    recommendations = df.iloc[top_indices][['Restaurant Name', 'City', 'Cuisines', 'Price range']]
    print(f"\nTop {top_n} Recommendations for you:\n")
    print(recommendations.to_string(index=False))

In [10]:
recommend_restaurants(user_cuisine="North Indian", user_city="Delhi", user_price_range=3)


Top 5 Recommendations for you:

          Restaurant Name      City     Cuisines  Price range
              Food Corner new delhi north indian            1
Sunny Restaurant & Tiffin new delhi north indian            1
         Mikky Restaurant new delhi north indian            2
        Shree Laxmi Dhaba new delhi north indian            1
         Shree Bhojnalaya new delhi north indian            1
